In [8]:
# Titanic 生还预测项目 - 步骤 1：加载数据
import pandas as pd
import numpy as np
# 读取训练集和测试集
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# 显示前几行数据
print("训练集预览：")
print(train_df.head())

print("\n测试集预览：")
print(test_df.head())

训练集预览：
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN       

In [ ]:
# ✅ 第三步：数据清洗与预处理

# 这一步我们会做以下几个重要操作：
# 	1.	填补缺失值（如年龄、登船港口等）
# 	2.	转换类别特征为数值（如性别、登船港口）
# 	3.	创建一些新特征（如家庭成员数量）

✅ 说明：
	•	将类别数据转换为数值是机器学习的必要操作；
	•	新建特征 FamilySize 可以帮助模型更好理解家庭对生还率的影响；
	•	后面我们会选取这些清洗后的特征来训练模型。

In [2]:
# 步骤 2：数据清洗与特征工程

import numpy as np

# 合并训练集和测试集一起处理
	# •	把 train_df 和 test_df 合并为一个大的 DataFrame：full_df。
	# •	这样做的目的是：在处理缺失值、特征转换时可以统一标准。
	# •	sort=False 表示列顺序不会被打乱。
full_df = pd.concat([train_df, test_df], sort=False)

# 1. 填补 Age（年龄）缺失值：用中位数填补
	# •	把 Age 中的 NaN 用中位数填补。
	# •	fillna() 是缺失值填充函数。
	# •	中位数比均值更稳定，不容易被异常值影响。
full_df["Age"] = full_df["Age"].fillna(full_df["Age"].median())

# 2. 填补 Embarked（登船港口）缺失值：用众数填补
full_df["Embarked"] = full_df["Embarked"].fillna(full_df["Embarked"].mode()[0])

# 3. 填补 Fare（票价）缺失值：用中位数
full_df["Fare"] = full_df["Fare"].fillna(full_df["Fare"].median())

# 4. 转换 Sex 为数字（male: 0, female: 1）
	# •	这是典型的 标签编码（Label Encoding）。
full_df["Sex"] = full_df["Sex"].map({"male": 0, "female": 1})

# 5. 转换 Embarked 为数字（S:0, C:1, Q:2）
	# •	将 Embarked 列中的字符串类别变为数值（One-Hot 的一种简单变体）。
full_df["Embarked"] = full_df["Embarked"].map({"S": 0, "C": 1, "Q": 2})

# 6. 新特征：家庭成员数量
	# •	构造新特征 FamilySize = 兄弟姐妹数 + 父母孩子数 + 本人。
	# •	是 Titanic 比赛中经典的工程特征之一，能反映是否独行或家庭同行。
full_df["FamilySize"] = full_df["SibSp"] + full_df["Parch"] + 1

# 检查处理后的结果
print("【处理后数据预览】")
print(full_df[["Sex", "Embarked", "Age", "Fare", "FamilySize"]].head())

【处理后数据预览】
   Sex  Embarked   Age     Fare  FamilySize
0    0         0  22.0   7.2500           2
1    1         1  38.0  71.2833           2
2    1         0  26.0   7.9250           1
3    1         0  35.0  53.1000           2
4    0         0  35.0   8.0500           1


In [ ]:
# ✅ 第四步：划分训练数据并训练第一个模型（逻辑回归）

# 🎯 目标：
# 	1.	从清洗后的数据中提取训练集和测试集
# 	2.	选择特征列用于建模
# 	3.	使用逻辑回归模型进行训练
# 	4.	在训练集上进行预测和评估

In [9]:
# 步骤 3：提取特征并训练逻辑回归模型

# •	LogisticRegression：逻辑回归模型
# •	accuracy_score：用于计算预测的准确率
# •	train_test_split：将训练数据进一步分成训练和验证部分
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# 回到原始的训练集与测试集划分
	# •	将你之前合并后的 full_df 拆分回原始的 train_df 和 test_df，用于分别训练和预测。
train_cleaned = full_df[:len(train_df)]
test_cleaned = full_df[len(train_df):]

# 特征选择（你可以尝试添加更多特征）
features = ["Pclass", "Sex", "Age", "Fare", "Embarked", "FamilySize"]

# •	X：用于训练的自变量（输入）
# •	y：用于训练的因变量（输出，目标）
# •	X_test：未来要预测的数据（提交用）
X = train_cleaned[features]  # 特征数据
y = train_cleaned["Survived"]  # 标签数据（预测目标）
X_test = test_cleaned[features]  # 测试集数据（要预测）

# 拆分训练集：80%训练，20%验证
# •	random_state=42 保证每次划分一样（可复现）。
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 建立模型并训练
# •	max_iter=200：逻辑回归默认最大迭代次数是 100，如果数据复杂可能不收敛，设 200 更保险。
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

# 在验证集上预测
# ➡️ 用训练好的模型在验证集上做预测，并计算预测结果的准确率（accuracy）。
# •	如果准确率不错（比如 > 0.75），说明模型学得还可以；
# •	准确率太低则可能需要改进特征、处理方式、换更复杂模型。
val_preds = model.predict(X_val)
acc = accuracy_score(y_val, val_preds)
print(f"验证集准确率：{acc:.4f}")
# 这是对变量 acc 的格式化输出控制：
# •	acc 是我们计算出的准确率（accuracy），比如 0.7821234
# •	:.4f 的含义是：
# •	: 表示你要开始格式控制
# •	.4 表示保留 小数点后4位
# •	f 表示以 浮点数（float） 的格式显示

验证集准确率：0.8101


In [10]:
# ✅ 第五步：使用模型预测 test.csv，并生成提交文件上传到 Kaggle 📤
# 🎯 目标：
# 	•	用你训练好的模型，对 test.csv 乘客进行生还预测
# 	•	把预测结果保存为 Kaggle 要求的 submission.csv 格式
# 	•	上传到比赛页面查看得分！

In [11]:
# 步骤 4：对 test.csv 做预测并生成提交文件

# 对测试集进行预测（注意：这是 Kaggle 给你的数据，没有真实标签）
test_preds = model.predict(X_test)

# 转换为整数类型
test_preds = test_preds.astype(int)

# 构建提交文件格式（必须包含 PassengerId 和 Survived 两列）
submission = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": test_preds
})

# 保存为 CSV 文件，准备提交
submission.to_csv("submission.csv", index=False)

print("✅ 已保存为 submission.csv，可上传至 Kaggle 提交。")

✅ 已保存为 submission.csv，可上传至 Kaggle 提交。
